# Evaluate Models - Simulations 1 and 2

Evaluate both GNN-Siamese and Spectral Radius models

--------------------

### Imports

In [ ]:
from src.Simulation1 import *
from src.Simulation2 import *
from src.utils import GenerateSimulationPlot

---------

## Main

In [ ]:
generate_plot = GenerateSimulationPlot()

-----

## Simulation1



### Define Simulation 1 parameters

Must be a existing parameter combination, i.e. graph data generation must have been run before this.

In [ ]:
simulation_name="simulation_1" #Simulation name to be used on inputs dir.
graph_name_list=["erdos", "geometric","watts_strogatz","k_regular","barabasi"] #Graphs names to be generated
n_graphs=40 #Number of graphs in each set for each simulation
n_nodes=100 #Size of each graphs.
classes_comb = [['erdos','erdos'],['geometric','geometric'],['k_regular','k_regular'],['watts_strogatz','watts_strogatz'],['barabasi','barabasi'],
                ['erdos','geometric'],['erdos','k_regular'],['erdos','watts_strogatz'],['erdos','barabasi'],
                ['geometric','k_regular'],['geometric','watts_strogatz'],['geometric','barabasi'],
                ['k_regular','watts_strogatz'], ['k_regular','barabasi'],
                ['watts_strogatz','barabasi']]

In [ ]:
df_results = pd.DataFrame()
for graph_name in graph_name_list:
    evaluation = Simulation1Evaluation(n_nodes,n_graphs,graph_name)
    evaluation.read_test_data()

    #Simulate spectral radius
    spectral_corr_list,real_corr_list = evaluation.evaluate_spectral()
    len_df = len(real_corr_list)
    data_eig = {'real_corr':real_corr_list,'corr':spectral_corr_list,'graph_name':[graph_name]*len_df,'method':['spectral_radius']*len_df}
    df_results = df_results = pd.concat([df_results,pd.DataFrame(data=data_eig)])

    #Simulate GNN-Siamese
    evaluation.fit_test_data()
    gnn_siamese_corr_list,real_corr_list = evaluation.evaluate_GNN_Siamese()
    data_net = {'real_corr':real_corr_list,'corr':gnn_siamese_corr_list,'graph_name':[graph_name]*len_df,'method':['GNN_Siamese']*len_df}
    df_results = df_results = pd.concat([df_results,pd.DataFrame(data=data_net)])

df_results.to_csv(f'/graph_correlation/results/{simulation_name}_{n_nodes}_nodes_{n_graphs}_graphs_results.csv')

In [ ]:
# Plot regression
generate_plot.plot_sim1_regression(df_results,n_graphs,n_nodes)

-------
## Simulation 2


### Define Simulation 2 parameters

Must be a existing parameter combination, i.e. graph data generation must have been run before this.

In [ ]:
simulation_name="simulation_2a" #Simulation name to be used on inputs dir.
graph_name_list=[['erdos','erdos'],['geometric','geometric'],['k_regular','k_regular'],['watts_strogatz','watts_strogatz'],['barabasi','barabasi']] #Graphs names to be generated
n_graphs_sim2= [20,40,60,80,100] #Number of graphs in each set for each simulation
n_nodes= 50 #Size of each graphs.
classes_comb = [['erdos','erdos'],['geometric','geometric'],['k_regular','k_regular'],['watts_strogatz','watts_strogatz'],['barabasi','barabasi'],
                ['erdos','geometric'],['erdos','k_regular'],['erdos','watts_strogatz'],['erdos','barabasi'],
                ['geometric','k_regular'],['geometric','watts_strogatz'],['geometric','barabasi'],
                ['k_regular','watts_strogatz'], ['k_regular','barabasi'],
                ['watts_strogatz','barabasi']]

#### Spectral Radius

In [ ]:
df_all_sr_results = pd.DataFrame()
for n_graphs in n_graphs_sim2:
  #Get eigenvalues for all data
  df_eig_list = pd.DataFrame()
  for graph_names in graph_name_list:
    eval = Simulation2Evaluation(n_nodes=n_nodes,n_graphs=n_graphs,graph_names=graph_names,simulation_name=simulation_name)
    eval.read_test_data()
    df_eig_graph = eval.evaluate_spectral()
    df_eig_list = pd.concat([df_eig_list,df_eig_graph])

  df_eig_p_value = eval.get_p_value_from_spectral(df_eig_list,classes_comb)
  df_all_sr_results = pd.concat([df_all_sr_results,df_eig_p_value])
df_all_sr_results.to_csv(f'/graph_correlation/results/{simulation_name}_spectral_radius_results.csv')


In [ ]:
df_all_sr_results.head()

In [ ]:
# Plot ROC curves
generate_plot.plot_sim2_roc_curves(df_all_sr_results,simulation_name)

#### GNN-Siamese

In [ ]:
study_best_params = {'num_conv_hidden_layers': 2, 'conv_hidden_channels_0': 12, 'conv_hidden_channels_1': 9, 'n_graphs': 40, 'batch_size': 40, 'lr':0.0006987472238364155, 'activation': 'tanh', 'criterion': 'L1Loss', 'num_epochs': 30, 'gat_heads': 1, 'readout_layer': 'gadp'}

##GNN Siamese Evaluation:
df_all_gnn_results = pd.DataFrame()
for n_graphs in n_graphs_sim2:
  df_net_results = pd.DataFrame()
  for graph_names in classes_comb:
    eval = Simulation2Evaluation(n_nodes=n_nodes,n_graphs=n_graphs,graph_names=graph_names,simulation_name=simulation_name)
    eval.read_test_data()
    eval.fit_test_data()
    net_p_value_list = eval.evaluate_GNN_Siamese()
    results_len = len(net_p_value_list)
    df_net_results = pd.concat([df_net_results,pd.DataFrame(data={'p_value':net_p_value_list,'graph_pair':[graph_names]*results_len,'method':['GNN-Siamese']*results_len,'number_of_graphs':[n_graphs]*results_len})])
  df_all_gnn_results = pd.concat([df_all_gnn_results,df_net_results])
df_all_gnn_results.to_csv(f'/graph_correlation/results/{simulation_name}_gnn_siamese_results.csv')


In [ ]:
# Plot ROC curves
generate_plot.plot_sim2_roc_curves(df_all_gnn_results,simulation_name)

### Compare Power of the test between Spectral Radius and GNN-Siamese

In [ ]:
generate_plot.plot_compare_power(df_all_sr_results,df_all_gnn_results)